In [1]:
import os
import csv
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from keybert import KeyBERT
from PyDictionary import PyDictionary
import yake
from keybert import KeyBERT

In [2]:
nltk.download("stopwords")
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /Users/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
nlp = spacy.load('en_core_web_sm')

In [11]:
corpus = pd.read_csv("/Users/alex/Desktop/ВШЭ/Python/Project/df-corpus.csv", sep='\t')

Ключевые слова

In [10]:
def yake_keywords(text):
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 7
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text.lower())
    keywords1 = []
    for kw in keywords:
        keywords1.append(kw[0])
    lemmatized_keywords = ' '.join(keywords1)
    doc = nlp(lemmatized_keywords)
    lemmatized_keywords = [i.lemma_ for i in doc]
    lemmatized_keywords = list(set(lemmatized_keywords))     
    return lemmatized_keywords

Ключевые слова со словарными значениями

In [112]:
def keywords_with_definitions(text):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None)
    keybert_keywords = [i[0] for i in keywords]
    yake = yake_keywords(text)
    all_keywords = keybert_keywords + yake
    lemmatized_keywords = ' '.join(all_keywords)
    doc = nlp(lemmatized_keywords)
    lemmatized_keywords = [i.lemma_ for i in doc]
    lemmatized_keywords = list(set(lemmatized_keywords))
    dictionary = PyDictionary()
    definitions = [dictionary.meaning(i, disable_errors=True) for i in lemmatized_keywords]
    words_with_definitions = dict(zip(lemmatized_keywords,definitions))
    strings = []
    for key,item in words_with_definitions.items():
        strings.append("{} - {}".format(key.upper(), item))
    result = "\n".join(strings)
    return result

Саммари

In [190]:
def summary(text):
    docx = nlp(text)
    mytokens = [token.text for token in docx]
    word_frequencies = {}
    for word in docx:
        if word.text not in stopwords:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
    maximum_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
    sentence_list = [sentence for sentence in docx.sents]
    sentence_scores = {}
    for sent in sentence_list:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if len(sent.text.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word.text.lower()]
                    else:
                        sentence_scores[sent] += word_frequencies[word.text.lower()]

    list_of_sentences = list(sentence_scores)
    summary = ""
    for i in list_of_sentences:
        summary += str(i)+ " " 
    return summary

In [ ]:
corpus['keywords with definitions'] = corpus['texts'].apply(keywords_with_definitions)

In [ ]:
corpus['summary'] = corpus['texts'].apply(summary)

In [3]:
display(corpus)

NameError: name 'corpus' is not defined